# Set up

In [1]:
!pip list

Package                   Version
------------------------- --------------
annotated-types           0.7.0
anyio                     4.9.0
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 3.0.0
async-lru                 2.0.5
attrs                     25.3.0
babel                     2.17.0
beautifulsoup4            4.13.3
bleach                    6.2.0
certifi                   2025.1.31
cffi                      1.17.1
charset-normalizer        3.4.1
comm                      0.2.2
contourpy                 1.3.1
cryptography              44.0.2
cycler                    0.12.1
debugpy                   1.8.13
decorator                 5.2.1
defusedxml                0.7.1
dill                      0.3.9
executing                 2.2.0
fastjsonschema            2.21.1
fonttools                 4.56.0
fqdn                      1.5.1
h11                       0.14.0
httpcore                  1.0.7
httpx        

#### Verify that the setup was done correctly

In [2]:
import qiskit
print(qiskit.__version__)

1.4.2


### Setting up the quantum service

#### Imports

In [4]:
from qiskit import QuantumCircuit, transpile
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Session
from qiskit.visualization import plot_histogram

In [7]:
service = QiskitRuntimeService(
    channel="ibm_quantum",
    token="7821f65d0dd6b850e5efee1028ddc389a6b680756509dd57ee3ce5485f0b7063ef495a3ce96da2ae386537a1734b63fb229452cbdbf7e6146cca3909257b78ab",
)

service.backends()

[<IBMBackend('ibm_brisbane')>,
 <IBMBackend('ibm_kyiv')>,
 <IBMBackend('ibm_sherbrooke')>]

In [ ]:
backend = service.backend(name="ibm_kyiv")
print(f"Number of Qubits: {backend.num_qubits}")

Number of Qubits: 127


# Deutsch Josza Algorithm
The Deutsch Jpsza Algorithm detemines if a given function $f(x)$ is constant or balanced with a quantum approach
Steps:
1. Start with state $\ket{01} = \ket{0} \otimes \ket{1}$
1. Apply H-gate transformation H to both bits
1. Apply function $f$
1. Apply H-tranform to both bits
1. Measure the firest bit of the resulting 2-bit state:
    * If the first bit is 0, we conclude that function $f$ is constant
    * If the first bit is 1, we conclude that function $f$ is not constant


##### Imports

In [9]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.primitives import StatevectorSampler
from qiskit.visualization import plot_histogram
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager


#### Oracle (Black Box)
The Oracle is the black box in this algorithm that takes an input $x$ and flips the output qubit if $f(x) = 1$.

There are 2 types of oracles:
1. Constant Oracle: For a constant function (always 0 or always 1), the oracle does nothing if always 0 or applies x gate to output qubit if always 1.
1. Balanced Oracle: for a balanced function we can use a CNOT gate controlled by one of the input qubits

In [ ]:
def oracle(
    n: int, oracle_type: str = "constant", constant_value: int = 0
) -> QuantumCircuit:
    """
    Creates an oracle for the DJ Algorithm

    Args:
        n (int): Number of input qubits.
        oracle_type (str): Type of oracle ('constant' or 'balanced')
        constant_value (int): Value for constant oracle (0 or 1)

    Returns:
        QuantumCircuit: The oracle circuit
    """
    oracle = QuantumCircuit(n + 1)  # n input qubits plus the 1 output qubit

    if oracle_type == "constant":
        if constant_value == 1:
            oracle.x(n)  # Flip the output qubit to always output 1
    elif oracle_type == "balanced":
        for i in range(n):
            oracle.cx(i, n)
    oracle.name = f"{oracle_type}_oracle"
    return oracle

In [ ]:
def deutsch_josza_algorithm(n, oracle_type="constant", constant_value=0):
    pass